### Import Needed Library

In [1]:
import pandas as pd
import tensorflow as tf
import itertools
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

### Open the Dataset

In [2]:
df = pd.read_excel('dataset_herb.xlsx')

### See the Information of the dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   gejala             1001 non-null   object
 1   relevance_score    1001 non-null   object
 2   nama_herb          1001 non-null   object
 3   nama_latin         999 non-null    object
 4   nama_lokal         999 non-null    object
 5   link_gambar        1001 non-null   object
 6   deskripsi_khasiat  999 non-null    object
 7   mengatasi_gejala   999 non-null    object
 8   komposisi          999 non-null    object
 9   cara_menggunakan   1001 non-null   object
dtypes: object(10)
memory usage: 78.3+ KB


### Change Score Column Datatype to Float

In [4]:
df['relevance_score'] = pd.to_numeric(df['relevance_score'].astype(str).str.replace(',', '.'), errors='coerce')
df

,gejala,relevance_score,nama_herb,nama_latin,nama_lokal,link_gambar,deskripsi_khasiat,mengatasi_gejala,komposisi,cara_menggunakan
0,Batuk,0.7,Adas,Foeniculum vulgare,"Hades (Sunda, Bali); Adas Londa, Adas Landi (J...",https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman rempah yang memiliki aroma khas dan se...,Batuk\nSesak nafas\nSariawan\nHaid tidak terat...,"Minyak atsiri (anethole, fenchone, estragole),...","Rebus 1-2 sendok teh biji adas dalam 1,5 gelas..."
1,Sakit tenggorokan,0.6,Adas,Foeniculum vulgare,"Hades (Sunda, Bali); Adas Londa, Adas Landi (J...",https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman rempah yang memiliki aroma khas dan se...,Batuk\nSesak nafas\nSariawan\nHaid tidak terat...,"Minyak atsiri (anethole, fenchone, estragole),...",Rebus 1-2 sendok teh biji adas dalam segelas a...
2,Sariawan,0.8,Adas,Foeniculum vulgare,"Hades (Sunda, Bali); Adas Londa, Adas Landi (J...",https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman rempah yang memiliki aroma khas dan se...,Batuk\nSesak nafas\nSariawan\nHaid tidak terat...,"Minyak atsiri (anethole, fenchone, estragole),...",Rebus 1-2 sendok teh biji adas dalam segelas a...
3,Nyeri haid,0.9,Adas,Foeniculum vulgare,"Hades (Sunda, Bali); Adas Londa, Adas Landi (J...",https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman rempah yang memiliki aroma khas dan se...,Batuk\nSesak nafas\nSariawan\nHaid tidak terat...,"Minyak atsiri (anethole, fenchone, estragole),...",Rebus 1-2 sendok teh biji adas dalam segelas a...
4,Perut kembung,1.0,Adas,Foeniculum vulgare,"Hades (Sunda, Bali); Adas Londa, Adas Landi (J...",https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman rempah yang memiliki aroma khas dan se...,Batuk\nSesak nafas\nSariawan\nHaid tidak terat...,"Minyak atsiri (anethole, fenchone, estragole),...",Seduh 1-2 sendok teh biji adas dalam segelas a...
...,...,...,...,...,...,...,...,...,...,...
996,Mata lelah,1.0,Wortel,Daucus carota,Wortel Jepang (Bali); Wortel Kuning (Sumatera),https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman umbi yang kaya akan beta-karoten dan s...,Mata lelah\nKulit kusam\nPerut kembung\nTenggo...,"Beta-karoten, vitamin A, vitamin C, kalium, serat","Kupas dan cuci bersih 2-3 wortel segar, lalu p..."
997,Kulit kusam,0.8,Wortel,Daucus carota,Wortel Jepang (Bali); Wortel Kuning (Sumatera),https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman umbi yang kaya akan beta-karoten dan s...,Mata lelah\nKulit kusam\nPerut kembung\nTenggo...,"Beta-karoten, vitamin A, vitamin C, kalium, serat","Rebus 2 wortel ukuran sedang hingga empuk, lal..."
998,Perut kembung,0.6,Wortel,Daucus carota,Wortel Jepang (Bali); Wortel Kuning (Sumatera),https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman umbi yang kaya akan beta-karoten dan s...,Mata lelah\nKulit kusam\nPerut kembung\nTenggo...,"Beta-karoten, vitamin A, vitamin C, kalium, serat",Iris tipis 2 wortel segar dan rebus dalam 2 ge...
999,Tenggorokan kering,0.7,Wortel,Daucus carota,Wortel Jepang (Bali); Wortel Kuning (Sumatera),https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman umbi yang kaya akan beta-karoten dan s...,Mata lelah\nKulit kusam\nPerut kembung\nTenggo...,"Beta-karoten, vitamin A, vitamin C, kalium, serat",Blender 1-2 wortel dengan setengah gelas air h...


### Separate the Data for Model and Data for Further Information

In [5]:
df_raw_data = df.iloc[:, :3]
df_information = df.iloc[:, 2:]
df_information = df_information.drop_duplicates(subset='nama_herb', keep='first')

### See the Data for Modelling

In [6]:
df_raw_data

,gejala,relevance_score,nama_herb
0,Batuk,0.7,Adas
1,Sakit tenggorokan,0.6,Adas
2,Sariawan,0.8,Adas
3,Nyeri haid,0.9,Adas
4,Perut kembung,1.0,Adas
...,...,...,...
996,Mata lelah,1.0,Wortel
997,Kulit kusam,0.8,Wortel
998,Perut kembung,0.6,Wortel
999,Tenggorokan kering,0.7,Wortel


### Construct a Matrix for Representation and Easier Processing Further

In [7]:
df_data = df_raw_data.pivot_table(index='gejala', columns='nama_herb', values='relevance_score', fill_value=0).reset_index()
df_data

nama_herb,gejala,Adas,Akar bajakah,Akar manis,Akar wangi,Alang-alang,Andong,Aren,Asam jawa,Bawang merah,...,Teratai Kerdil,Tomat,Tunjung,Turi,Ubi Kayu,Urang Aring,Waru,Wijaya Kusuma,Wortel,Zaitun
0,Batuk,0.7,0.00,0.80,0.00,0.6,0.00,0.6,0.6,0.6,...,0.0,0.0,0.0,0.8,0.00,0.00,0.8,0.7,0.0,0.00
1,Bau badan,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
2,Bau mulut,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
3,Darah tinggi,0.0,0.00,0.00,0.00,0.7,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
4,Dehidrasi,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,...,0.0,0.9,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
5,Demam,0.0,0.80,0.00,0.00,0.0,0.00,0.5,0.8,0.8,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
6,Diare,0.7,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.85,0.00,0.0,0.0,0.0,0.00
7,Gangguan pencernaan,0.0,0.00,0.90,0.00,1.0,0.75,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
8,Gusi berdarah,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
9,Iritasi kulit,0.0,0.00,0.00,0.00,0.0,0.85,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00


### Encode Categorical Value

In [8]:
gejala_encoder = OrdinalEncoder()
df_data['gejala'] = gejala_encoder.fit_transform(df_data[['gejala']])

print(dict(zip(gejala_encoder.categories_[0], range(len(gejala_encoder.categories_[0])))))

{'Batuk': 0, 'Bau badan': 1, 'Bau mulut': 2, 'Darah tinggi': 3, 'Dehidrasi': 4, 'Demam': 5, 'Diare': 6, 'Gangguan pencernaan': 7, 'Gusi berdarah': 8, 'Iritasi kulit': 9, 'Jerawat': 10, 'Kelelahan': 11, 'Kolesterol': 12, 'Kram perut': 13, 'Kulit berminyak': 14, 'Kulit gatal': 15, 'Kulit kasar': 16, 'Kulit kering': 17, 'Kulit kusam': 18, 'Lemas': 19, 'Luka': 20, 'Luka bakar ringan': 21, 'Mata lelah': 22, 'Mimisan': 23, 'Mual': 24, 'Nyeri haid': 25, 'Nyeri otot': 26, 'Nyeri sendi': 27, 'Panas dalam': 28, 'Pembengkakan': 29, 'Perut kembung': 30, 'Pilek': 31, 'Pusing': 32, 'Sakit gigi': 33, 'Sakit kepala': 34, 'Sakit perut': 35, 'Sakit tenggorokan': 36, 'Sariawan': 37, 'Sembelit': 38, 'Stres': 39, 'Susah tidur': 40, 'Tenggorokan gatal': 41, 'Tenggorokan kering': 42, 'Tidak nafsu makan': 43}


### Split Input (X - Symptom) from Output (Y - Array of Herb Relevance Score)

In [9]:
X = df_data.iloc[:, 0]
X

0      0.0
1      1.0
2      2.0
3      3.0
4      4.0
5      5.0
6      6.0
7      7.0
8      8.0
9      9.0
10    10.0
11    11.0
12    12.0
13    13.0
14    14.0
15    15.0
16    16.0
17    17.0
18    18.0
19    19.0
20    20.0
21    21.0
22    22.0
23    23.0
24    24.0
25    25.0
26    26.0
27    27.0
28    28.0
29    29.0
30    30.0
31    31.0
32    32.0
33    33.0
34    34.0
35    35.0
36    36.0
37    37.0
38    38.0
39    39.0
40    40.0
41    41.0
42    42.0
43    43.0
Name: gejala, dtype: float64

In [10]:
Y = df_data.iloc[:, 1:]
Y

nama_herb,Adas,Akar bajakah,Akar manis,Akar wangi,Alang-alang,Andong,Aren,Asam jawa,Bawang merah,Bawang putih,...,Teratai Kerdil,Tomat,Tunjung,Turi,Ubi Kayu,Urang Aring,Waru,Wijaya Kusuma,Wortel,Zaitun
0,0.7,0.00,0.80,0.00,0.6,0.00,0.6,0.6,0.6,0.00,...,0.0,0.0,0.0,0.8,0.00,0.00,0.8,0.7,0.0,0.00
1,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
2,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
3,0.0,0.00,0.00,0.00,0.7,0.00,0.0,0.0,0.0,0.95,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
4,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.9,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
5,0.0,0.80,0.00,0.00,0.0,0.00,0.5,0.8,0.8,0.00,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
6,0.7,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.85,0.00,0.0,0.0,0.0,0.00
7,0.0,0.00,0.90,0.00,1.0,0.75,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
8,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00
9,0.0,0.00,0.00,0.00,0.0,0.85,0.0,0.0,0.0,0.50,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00


### Train the Model with Data

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(44, activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(197, activation='relu')
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.fit(X, Y, epochs=100, batch_size=1, verbose=1)



Epoch 1/100


44/44 [==============================] - 1s 5ms/step - loss: 0.0755 - mae: 0.0937
Epoch 2/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0644 - mae: 0.0849
Epoch 3/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0643 - mae: 0.0849
Epoch 4/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0642 - mae: 0.0849
Epoch 5/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0640 - mae: 0.0846
Epoch 6/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0643 - mae: 0.0849
Epoch 7/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0643 - mae: 0.0848
Epoch 8/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0641 - mae: 0.0846
Epoch 9/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0639 - mae: 0.0845
Epoch 10/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0638 - mae: 0.0842
Epoch 11/100
44/44 [==============================] - 0

44/44 [==============================] - 0s 5ms/step - loss: 0.0607 - mae: 0.0956
Epoch 79/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0610 - mae: 0.0963
Epoch 80/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0608 - mae: 0.0949
Epoch 81/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0608 - mae: 0.0957
Epoch 82/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0608 - mae: 0.0964
Epoch 83/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0607 - mae: 0.0961
Epoch 84/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0606 - mae: 0.0959
Epoch 85/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0605 - mae: 0.0954
Epoch 86/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0604 - mae: 0.0957
Epoch 87/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0605 - mae: 0.0969
Epoch 88/100
44/44 [==============================] - 0s 5ms/st

### Do the Prediction (for example, for Symptom 0: Batuk)

In [12]:
Y_predict = model.predict([[0.0]])
print(Y_predict)

1/1 [==============================] - 0s 111ms/step
[[0.         0.         0.         0.         0.         0.
  0.21850744 0.23806089 0.         0.         0.         0.
  0.1308543  0.6509492  0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.27800098 0.         0.         0.         0.         0.5792721
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.17406902 0.         0.56283987 0.         0.
  0.         0.         0.         0.         0.         0.69251263
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.      

In [13]:
df_information.sort_values(by='nama_herb', inplace=True)

In [14]:
import numpy as np

index = [
    col for col in range(Y_predict.shape[1]) if np.any(Y_predict[:, col] > 0.1)
]

df_information.iloc[index, :]

,nama_herb,nama_latin,nama_lokal,link_gambar,deskripsi_khasiat,mengatasi_gejala,komposisi,cara_menggunakan
24,Aren,Arenga pinnata,"Aren (Jawa, Sunda); Bagot (Batak); Bilulu (Nus...",https://drive.google.com/file/d/1xoq7gHvGC4FaA...,pohon palem yang dikenal sebagai sumber gula a...,Batuk\nSembelit\nDemam\nAnemia\nPerut kembung,"Sukrosa, glukosa, kalsium, magnesium, fosfor, ...",Gunakan nira aren yang segar. Minum 1-2 sendok...
29,Asam jawa,Tamarindus indica,Asem (Jawa); Bakasap (Aceh); Koseka (Sulawesi)...,https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman yang dikenal dengan buahnya yang memil...,Demam\nBatuk\nSembelit\nSakit tenggorokan\nNye...,"Asam tartarat, pektin, flavonoid, saponin, tan...",Asam jawa dapat membantu menurunkan demam. Reb...
49,Belimbing,Averrhoa carambola,Blimbing Manis (Jawa); Karuntung (Sulawesi); B...,https://drive.google.com/file/d/1xoq7gHvGC4FaA...,pohon buah yang sering ditanam di pekarangan r...,Demam\nDarah tinggi\nSembelit\nBatuk\nJerawat,"Vitamin C, kalium, flavonoid, serat, oksalat.",Belimbing memiliki sifat antipiretik yang memb...
54,Belimbing wuluh,Averrhoa bilimbi,"Belimbing Asam (Jawa, Sunda); Asam Belimbing (...",https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman kecil dengan buah berbentuk lonjong be...,Batuk\nSakit tenggorokan\nDarah tinggi\nJerawa...,"Asam oksalat, vitamin C, saponin, tanin, flavo...","Ambil 5-6 buah belimbing wuluh, cuci bersih, l..."
109,Ciplukan,Physalis angulata,Cecenet (Sunda); Kopok-kopokan (Jawa); Cipluk ...,https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman semak kecil dengan buah berbentuk bula...,Demam\nBatuk\nSakit tenggorokan\nNyeri sendi\n...,"Vitamin C, flavonoid, saponin, alkaloid, polif...",Rebus 10-15 daun ciplukan segar dalam 2 gelas ...
129,Daun jintan,Coleus amboinicus,Torbangun (Batak); Jintan Alas (Sunda); Daun J...,https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman semak kecil dengan daun berbentuk hati...,Perut kembung\nBatuk\nMual\nNyeri otot\nSakit ...,"Minyak atsiri, flavonoid, saponin, fenolik.",Rebus beberapa lembar daun jintan dalam segela...
179,Iler,Plectranthus scutellarioides,Miana (Sulawesi); Kentangan (Jawa); Oler (Sund...,https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman semak kecil dengan daun berwarna hijau...,Demam\nSariawan\nLuka\nBatuk\nPerut kembung,"Flavonoid, tanin, alkaloid, minyak atsiri.",Iler dapat membantu menurunkan demam dengan ca...
174,Jahe,Zingiber officinale,Jae (Jawa); Uwe Ala (Minangkabau); Balaka (Ace...,https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman rimpang dengan aroma khas dan rasa ped...,Mual\nBatuk\nPerut kembung\nNyeri otot\nPusing,"Gingerol, shogaol, minyak atsiri, vitamin C, f...","Jahe efektif meredakan mual, termasuk mual aki..."
204,Jeruk nipis,Citrus aurantiifolia,Jeruk Pecel (Jawa); Lime (Sunda); Limau Kecil ...,https://drive.google.com/file/d/1xoq7gHvGC4FaA...,pohon buah kecil dengan daun hijau lebat dan b...,Batuk\nMual\nPerut kembung\nSakit tenggorokan\...,"Vitamin C, asam sitrat, flavonoid, minyak atsiri.",Campurkan 1 sendok makan air perasan jeruk nip...
334,Legundi,Vitex trifolia,Ligundi (Jawa); Lugundi (Sulaawesi); Langgundi...,https://drive.google.com/file/d/1xoq7gHvGC4FaA...,tanaman obat yang daunnya sering digunakan dal...,Batuk\nNyeri otot\nLuka\nSakit tenggorokan\nKe...,"Minyak atsiri, flavonoid, iridoid.",Rebus 1 genggam daun legundi segar dalam 2 gel...


### Save model into H5, Keras, and TF model

In [15]:
model.save('herb_model.h5')

C:\Users\LENOVO\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
model.save('herb_model')

INFO:tensorflow:Assets written to: herb_model\assets


INFO:tensorflow:Assets written to: herb_model\assets


In [17]:
model.save('herb_model.keras')